In [2]:
import json
from tqdm import tqdm
from collections import Counter

In [3]:

def extract1205():
    '''
    提取训练集当中的文本问答对
    '''
    with open(r"E:\RecipeQA\data\数据集\r2vq_train_10_28_2021\train\crl_srl.csv", 'r', encoding='utf-8') as f:
        all_data = f.readlines()
    all_data2=[] #存储所有信息
    i=1
    while i<(219125):
        doc_id = all_data[i-1].split("=")[1].strip("\n") #获取doc_id
        list_2qa=[]   #2代表第二类问题，存储quest和answer
        text_tags=[]  #存储食谱句子和标签
        while 'newdoc id' not in  all_data[i]:
            item = all_data[i]
            item_next = all_data[i + 1]
            if 'question' in item and 'answer' in item_next:
                quest_idx = item.strip().split("=")[0].split(" ")[-2]
    #             if quest_idx.startswith('2'):
                if True:
                    temp_qa={}
                    quest = item.strip().split("=")[1]
                    answer = item_next.strip().split("=")[-1]
                    temp_qa['quest']=quest_idx+"="+quest
                    temp_qa['answer']=answer
                    list_2qa.append(temp_qa)
                i+=1
            elif 'text =' in item:
                item_pre = all_data[i - 1]
                if 'ingredients' in item_pre:
                    i+=1
                    pass
                else:
                    i+=1
                    temp_text_tags={}
                    texts,hiddens="",[]
                    while all_data[i] and "sent_id" not in all_data[i] and 'newdoc id' not in all_data[i]: #在数据集的最后一行加了newdoc id
                        if "newpar id" in all_data[i]:
                            i+=1
                            continue
                        item=all_data[i]

                        tags = item.strip().split("\t")
                        if len(tags)!=1:
                            word = tags[1]
                            texts+=(" "+word)
                        i+=1
                    temp_text_tags["text"]=texts
                    text_tags.append(temp_text_tags)
            else:
                i+=1
        i+=1

        doc_dict={}
        doc_dict['doc_id']=doc_id
        doc_dict['list_2qa']=list_2qa
        doc_dict['text_tags']=text_tags
        all_data2.append(doc_dict)
    fw = open("all_data.json", "w", encoding="utf-8")
    json.dump(all_data2, fw, ensure_ascii=False, indent=4)
    fw.close()

In [4]:
def extarct_EM():
    f = open(r"E:\RecipeQA\data\ITNLP_Semeval2022_Task6\fengmq_1205\all_data.json", 'r', encoding='utf-8')
    json_data = json.load(f)
    f.close()
    result=[]
    for item in tqdm(json_data):
        list_qa=item['list_2qa']
        text_tags = item['text_tags']
        text=""
        #合并生成最终的连续text
        for tt in text_tags:
            text+=tt['text'].lower()
        text = text.strip()
        #处理问答对
        for qa in list_qa:
            quest=qa['quest']
            ans = qa['answer'].strip()
            if not ans.isdigit() and ans in text:
                temp_dict={}
                temp_dict['qa']=qa
                temp_dict['text']=text_tags
                result.append(temp_dict)
    fw = open("EM_all_data.json", "w", encoding="utf-8")
    json.dump(result, fw, ensure_ascii=False, indent=4)
    fw.close()
    print(len(result))
extarct_EM()

100%|█████████████████████████████████████████████████████████████████████████████| 800/800 [00:00<00:00, 32084.63it/s]


8235


In [7]:
#统计精准匹配的分布
f = open(r"E:\RecipeQA\data\ITNLP_Semeval2022_Task6\fengmq_1205\EM_all_data.json", 'r', encoding='utf-8')
json_data = json.load(f)
f.close()
result=[]
for item in json_data:
    q=item['qa']['quest']
    idx = q.split("=")[0].split("-")[0]
    result.append(int(idx))

sorted(Counter(result).items(), key=lambda d: d[0], reverse=False)

[(1, 670),
 (2, 1629),
 (3, 140),
 (5, 801),
 (6, 201),
 (7, 824),
 (8, 1831),
 (9, 30),
 (10, 213),
 (11, 5),
 (12, 289),
 (13, 41),
 (14, 144),
 (15, 289),
 (16, 194),
 (17, 934)]